In [1]:
import pandas as pd
import numpy as np

In [2]:
happy = pd.read_csv('data/happy.csv')
freedom = pd.read_csv('data/freedom.csv')
suicides = pd.read_csv('data/suicide.csv')

In [3]:
happy = happy.dropna()

In [13]:
happy = happy.drop(columns=['Regional Indicator', 'Log GDP Per Capita', 'Healthy Life Expectancy At Birth', 'Generosity'])

,Country,Year,Life Ladder,Social Support,Freedom To Make Life Choices,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
0,Afghanistan,2008,3.723590,0.450662,0.718114,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,2009,4.401778,0.552308,0.678896,0.850035,0.481421,0.237092,0.611545
2,Afghanistan,2010,4.758381,0.539075,0.600127,0.706766,0.516907,0.275324,0.299357
3,Afghanistan,2011,3.831719,0.521104,0.495901,0.731109,0.479835,0.267175,0.307386
4,Afghanistan,2012,3.782938,0.520637,0.530935,0.775620,0.613513,0.267919,0.435440


In [14]:
suicides = suicides.drop(columns=['country-year', 'HDI for year'])
suicides = suicides.drop(columns=[col for col in suicides.columns if "gdp" in col])

,Country,Year,sex,age,suicides_no,population,suicides/100k pop,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Boomers


In [15]:
freedom = freedom.drop(columns=[col for col in freedom.columns if len(col) == 2 or len(col) == 1])
freedom = freedom.drop(columns=["Add Q", "Add A", 'C/T', 'Status', 'Region'])

,Country,Year,PR rating,CL rating,Total
0,Abkhazia,2023,5,5,39
1,Afghanistan,2023,7,7,8
2,Albania,2023,3,3,67
3,Algeria,2023,6,5,32
4,Andorra,2023,1,1,93


In [16]:
merged_df = pd.merge(happy, freedom, on=['Country', 'Year'], how='inner')
merged_df = pd.merge(merged_df, suicides, on=['Country', 'Year'], how='inner')

Index(['Country', 'Year', 'Life Ladder', 'Social Support',
       'Freedom To Make Life Choices', 'Perceptions Of Corruption',
       'Positive Affect', 'Negative Affect',
       'Confidence In National Government', 'PR rating', 'CL rating', 'Total'],
      dtype='object')

In [18]:
merged_df = merged_df.rename({'Total': 'Total freedom'})

In [20]:
merged_df = merged_df.rename(columns={"Total": "Total freedom"})

In [22]:
df = merged_df.copy()
df['weighted_suicides'] = df['suicides/100k pop'] * df['population']
    
grouped = df.groupby(['Country', 'Year'])
aggregated_data = grouped.agg({
    'population': 'sum',
    'suicides_no': 'sum',
    'Life Ladder': 'first',  # Assuming these values are constant for each group
    'Social Support': 'first',
    'Freedom To Make Life Choices': 'first',
    'Perceptions Of Corruption': 'first',
    'Positive Affect': 'first',
    'Negative Affect': 'first',
    'Confidence In National Government': 'first',
    'PR rating': 'first',
    'CL rating': 'first',
    'Total freedom': 'first',
    'weighted_suicides': 'sum'
})
aggregated_data['suicides/100k pop'] = aggregated_data['weighted_suicides'] / aggregated_data['population']
aggregated_data.drop('weighted_suicides', axis=1, inplace=True)
aggregated_data.reset_index(inplace=True)

# Add the constant columns
aggregated_data['generation'] = 'All'
aggregated_data['sex'] = 'Both'
aggregated_data['age'] = 'All'
sum_data = aggregated_data

In [23]:
grouped = df.groupby(['Country', 'Year', 'sex'])
aggregated_data = grouped.agg({
    'population': 'sum',
    'suicides_no': 'sum',
    'Life Ladder': 'first',  # Assuming these values are constant for each group
    'Social Support': 'first',
    'Freedom To Make Life Choices': 'first',
    'Perceptions Of Corruption': 'first',
    'Positive Affect': 'first',
    'Negative Affect': 'first',
    'Confidence In National Government': 'first',
    'PR rating': 'first',
    'CL rating': 'first',
    'Total freedom': 'first',
    'weighted_suicides': 'sum'
})
aggregated_data['suicides/100k pop'] = aggregated_data['weighted_suicides'] / aggregated_data['population']
aggregated_data.drop('weighted_suicides', axis=1, inplace=True)
aggregated_data.reset_index(inplace=True)

aggregated_data['generation'] = 'All'
aggregated_data['age'] = 'All'

In [24]:
new_df = pd.concat([merged_df, sum_data])
display(new_df.tail())
final_df = pd.concat([new_df, aggregated_data])
final_df.reset_index(inplace=True)

,Country,Year,Life Ladder,Social Support,Freedom To Make Life Choices,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,PR rating,CL rating,Total freedom,sex,age,suicides_no,population,suicides/100k pop,generation
169,United States,2014,7.151114,0.902097,0.866077,0.702267,0.785791,0.281265,0.348671,1,1,92,Both,All,42769,297749735,14.364913,All
170,United States,2015,6.863947,0.903571,0.848753,0.697543,0.768671,0.274688,0.346936,1,1,92,Both,All,44189,300078511,14.726873,All
171,Uruguay,2013,6.444465,0.917280,0.888278,0.585632,0.742916,0.253229,0.524400,1,1,97,Both,All,546,3164670,17.252481,All
172,Uruguay,2014,6.561444,0.901898,0.904333,0.533495,0.788126,0.251499,0.595346,1,1,98,Both,All,591,3177645,18.598660,All
173,Uruguay,2015,6.628080,0.891493,0.916880,0.673476,0.811902,0.299538,0.550559,1,1,98,Both,All,630,3190795,19.743488,All


In [25]:
final_df

,index,Country,Year,Life Ladder,Social Support,Freedom To Make Life Choices,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,PR rating,CL rating,Total freedom,sex,age,suicides_no,population,suicides/100k pop,generation
0,0,Argentina,2013,6.582260,0.909874,0.737250,0.822900,0.766157,0.254205,0.433749,2,2,80,male,75+ years,168,699046,24.030000,Silent
1,1,Argentina,2013,6.582260,0.909874,0.737250,0.822900,0.766157,0.254205,0.433749,2,2,80,male,15-24 years,696,3486661,19.960000,Millenials
2,2,Argentina,2013,6.582260,0.909874,0.737250,0.822900,0.766157,0.254205,0.433749,2,2,80,male,25-34 years,499,3271373,15.250000,Millenials
3,3,Argentina,2013,6.582260,0.909874,0.737250,0.822900,0.766157,0.254205,0.433749,2,2,80,male,55-74 years,398,2894129,13.750000,Boomers
4,4,Argentina,2013,6.582260,0.909874,0.737250,0.822900,0.766157,0.254205,0.433749,2,2,80,male,35-54 years,604,4950850,12.200000,Generation X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2579,343,Uruguay,2013,6.444465,0.917280,0.888278,0.585632,0.742916,0.253229,0.524400,1,1,97,male,All,423,1519761,27.832337,All
2580,344,Uruguay,2014,6.561444,0.901898,0.904333,0.533495,0.788126,0.251499,0.595346,1,1,98,female,All,131,1651079,7.933370,All
2581,345,Uruguay,2014,6.561444,0.901898,0.904333,0.533495,0.788126,0.251499,0.595346,1,1,98,male,All,460,1526566,30.133855,All
2582,346,Uruguay,2015,6.628080,0.891493,0.916880,0.673476,0.811902,0.299538,0.550559,1,1,98,female,All,131,1657298,7.903728,All


In [28]:
final_df["age"].value_counts()

age
All            522
15-24 years    348
75+ years      348
25-34 years    348
55-74 years    348
35-54 years    348
5-14 years     322
Name: count, dtype: int64